# Pyspark Dataframe Get Stat

In [19]:
import json
import numpy as np
import pandas as pd
from time import time

from pyspark import SQLContext
from pyspark.sql.functions import *

import re
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
from __future__ import division

In [20]:
import pyspark
pyspark.__version__

'2.1.0+hadoop2.7'

In [21]:
sc

## 1. Define variables

In [22]:
month_list = ['201608','201611','201702','201705']
flist = ['cnt_voice_in_fix_avg_6m', 'cnt_voice_out_fix_avg_6m', 'arpu_int_roaming_avg_hy', 'arpu_onnet_roaming_avg_hy', 'call_charge_avg_6m', 'call_charge_ind_6m', 'cnt_voice_out_avg_6m', 'arpu_voice_avg_hy', 'arpu_gprs_avg_hy', 'arpu_vas_avg_hy', 'ingoing_avg_6m', 'intercity_in_avg_6m', 'int_roam_in_avg_6m', 'sms_out_comp_num_avg_6m', 'arpu_wo_roam_avg_hy', 'onn_roam_out_time_6m', 'cnt_onn_r_in_call_avg_6m', 'cnt_onn_r_out_call_avg_6m', 'data_volume_roam_avg_6m', 'cnt_pp_change_6m', 'cnt_fin_block_6m', 'cnt_block_6m', 'cpa_ind', 'cnt_package_m', 'cpa_ind_avg_6m', 'minus_bal_day_avg_1m', 'minus_bal_day_avg_hy', 'conf_call_charge_ind_1m', 'bl_charge_ind_1m', 'bal_charge_ind_1m', 'pay_charge_ind_1m', 'package_ind_1m', 'charge_package_1m', 'conf_call_charge_avg_6m', 'bl_charge_avg_6m', 'bal_charge_avg_6m', 'pay_charge_avg_6m', 'pay_charge_ind_6m', 'charge_package_avg_6m', 'conf_call_charge_ind_6m', 'bl_charge_ind_6m', 'bal_charge_ind_6m', 'int_state_6m', 'cnt_roam_state_6m', 'roam_state_6m', 'payment_method_1m', 'equipment_type_1m', 'payment_method_6m', 'equipment_type_6m', 'direct_number', 'cnt_pp_change_1m', 'cnt_block_1m', 'cnt_mou_bel_out', 'cnt_mou_eur_out', 'cnt_mou_cis_out', 'cnt_mou_ukr_out']
ttable = 'scheme.table_name'
XlsFileName = 'ATS'

In [23]:
mmonth=0
print 'Current month: ', month_list[mmonth]
print 'Features count: ', len(flist)
print 'Source table: ', ttable
SCR = sqlContext.sql('select *\
                      from %s where month in (%s)' %(ttable, month_list[mmonth]))\
                .select(*flist)

Current month:  201608
Features count:  56
Source table:  bank_score_bee.agg_bank_scoring_monthly


In [24]:
%%time
SCR.persist(storageLevel=StorageLevel.MEMORY_AND_DISK)

CPU times: user 3.7 ms, sys: 863 µs, total: 4.56 ms
Wall time: 550 ms


DataFrame[cnt_voice_in_fix_avg_6m: double, cnt_voice_out_fix_avg_6m: double, arpu_int_roaming_avg_hy: double, arpu_onnet_roaming_avg_hy: double, call_charge_avg_6m: double, call_charge_ind_6m: int, cnt_voice_out_avg_6m: double, arpu_voice_avg_hy: double, arpu_gprs_avg_hy: double, arpu_vas_avg_hy: double, ingoing_avg_6m: double, intercity_in_avg_6m: double, int_roam_in_avg_6m: double, sms_out_comp_num_avg_6m: double, arpu_wo_roam_avg_hy: double, onn_roam_out_time_6m: double, cnt_onn_r_in_call_avg_6m: double, cnt_onn_r_out_call_avg_6m: double, data_volume_roam_avg_6m: double, cnt_pp_change_6m: int, cnt_fin_block_6m: double, cnt_block_6m: double, cpa_ind: int, cnt_package_m: int, cpa_ind_avg_6m: double, minus_bal_day_avg_1m: int, minus_bal_day_avg_hy: double, conf_call_charge_ind_1m: int, bl_charge_ind_1m: int, bal_charge_ind_1m: int, pay_charge_ind_1m: int, package_ind_1m: int, charge_package_1m: double, conf_call_charge_avg_6m: double, bl_charge_avg_6m: double, bal_charge_avg_6m: double

In [25]:
SCR.is_cached

True

## 2. get dummy list
Try to get dummy list using value_counts

In [12]:
%%time
rcount = 10 # top N
rec_count = SCR.count();
print ('Get top %s by count:' %rcount)
print ('Value, %pers')
dummyDF = []
for ncol in SCR.columns:
    colDummy = SCR.groupBy(ncol)\
                .count()\
                .sort('count',ascending=False)\
                .limit(rcount)\
                .toPandas()\
                
    dummyDF.append(
        pd.Series(
            zip(colDummy[ncol], np.round(100*colDummy['count']/rec_count,5)),
            name=ncol))

Get top 10 by count:
Value, %pers
CPU times: user 804 ms, sys: 134 ms, total: 938 ms
Wall time: 4min 44s


In [15]:
pd.DataFrame(dummyDF).T

,cnt_voice_in_fix_avg_6m,cnt_voice_out_fix_avg_6m,arpu_int_roaming_avg_hy,arpu_onnet_roaming_avg_hy,call_charge_avg_6m,call_charge_ind_6m,cnt_voice_out_avg_6m,arpu_voice_avg_hy,arpu_gprs_avg_hy,arpu_vas_avg_hy,ingoing_avg_6m,intercity_in_avg_6m,int_roam_in_avg_6m,sms_out_comp_num_avg_6m,arpu_wo_roam_avg_hy,onn_roam_out_time_6m,cnt_onn_r_in_call_avg_6m,cnt_onn_r_out_call_avg_6m,data_volume_roam_avg_6m,cnt_pp_change_6m,cnt_fin_block_6m,cnt_block_6m,cpa_ind,cnt_package_m,cpa_ind_avg_6m,minus_bal_day_avg_1m,minus_bal_day_avg_hy,conf_call_charge_ind_1m,bl_charge_ind_1m,bal_charge_ind_1m,pay_charge_ind_1m,package_ind_1m,charge_package_1m,conf_call_charge_avg_6m,bl_charge_avg_6m,bal_charge_avg_6m,pay_charge_avg_6m,pay_charge_ind_6m,charge_package_avg_6m,conf_call_charge_ind_6m,bl_charge_ind_6m,bal_charge_ind_6m,int_state_6m,cnt_roam_state_6m,roam_state_6m,payment_method_1m,equipment_type_1m,payment_method_6m,equipment_type_6m,direct_number,cnt_pp_change_1m,cnt_block_1m,cnt_mou_bel_out,cnt_mou_eur_out,cnt_mou_cis_out,cnt_mou_ukr_out
0,"(0.0, 36.44472)","(0.0, 33.13964)","(0.0, 96.14453)","(0.0, 75.02563)","(0.0, 87.99501)","(0, 87.99501)","(0.0, 13.58016)","(0.0, 26.53751)","(0.0, 39.26247)","(0.0, 13.3228)","(0.0, 15.54721)","(0.0, 29.72496)","(0.0, 97.61997)","(0.0, 44.47844)","(0.0, 4.57167)","(0.0, 70.80923)","(0.0, 68.69287)","(0.0, 68.69339)","(0.0, 100.0)","(0, 72.8396)","(0.0, 99.68577)","(0.0, 57.09592)","(0, 94.72456)","(2, 26.28707)","(0.0, 94.54651)","(0, 64.32046)","(0.0, 48.16081)","(0, 99.92561)","(0, 99.76421)","(0, 99.99358)","(0, 89.33857)","(0, 53.53104)","(0.0, 45.42054)","(0.0, 99.88196)","(0.0, 99.67201)","(0.0, 99.97441)","(0.0, 82.66101)","(0, 82.66101)","(0.0, 37.29981)","(0, 99.88196)","(0, 99.67201)","(0, 99.97441)","(, 81.17129)","(0, 94.10098)","(, 94.10098)","(Не задано, 54.09539)","(, 35.816)","(Не задано, 73.30765)","(Не задано, 41.9869)","(, 98.33322)","(0, 90.28054)","(0, 70.29509)","(0, 99.64113)","(0, 99.50256)","(0, 94.22946)","(0, 98.44872)"
1,"(0.1667, 7.92739)","(0.1667, 6.19354)","(14.125, 0.2669)","(1.405, 0.83449)","(0.1667, 3.91076)","(1, 12.00499)","(1.0, 0.90747)","(0.4233, 0.40765)","(127.1209, 1.16724)","(12.8499, 0.53787)","(0.0028, 0.05414)","(0.0333, 0.16718)","(0.0056, 0.01579)","(0.1667, 5.63027)","(254.24, 0.41787)","(1.0, 0.10079)","(0.1667, 3.51933)","(0.1667, 3.75152)",NaN,"(1, 23.84015)","(0.3333, 0.10331)","(1.0, 7.19978)","(1, 5.27544)","(1, 23.61381)","(1.0, 4.83096)","(31, 6.64394)","(0.1667, 2.74517)","(1, 0.07439)","(1, 0.23579)","(1, 0.00642)","(1, 10.66143)","(1, 46.46896)","(147.4606, 1.91014)","(1.0, 0.04861)","(1.0, 0.15268)","(0.1667, 0.01565)","(1.0, 3.90565)","(1, 17.33899)","(84.7511, 0.824)","(1, 0.11804)","(1, 0.32799)","(1, 0.02559)","(Узбекистан, 2.72539)","(1, 4.47098)","(Беларусь, 0.8011)","(, 35.816)","(Не задано, 30.61555)","(, 16.90326)","(, 16.90326)","(3512163516, 0.0)","(1, 9.46134)","(1, 20.05228)","(1, 0.14994)","(1, 0.24891)","(1, 0.85155)","(1, 0.41276)"
2,"(0.3333, 5.73272)","(0.3333, 4.59722)","(1.4117, 0.17053)","(2.81, 0.62025)","(0.3333, 2.05969)",NaN,"(0.1667, 0.76149)","(0.8467, 0.28577)","(0.2067, 0.7255)","(19.627, 0.48832)","(0.0333, 0.04089)","(0.05, 0.1631)","(0.1667, 0.01547)","(0.3333, 3.71231)","(8.47, 0.39849)","(0.3333, 0.06346)","(0.3333, 2.41152)","(0.3333, 2.46667)",NaN,"(2, 2.85313)","(0.1667, 0.09402)","(0.1667, 6.99988)",NaN,"(3, 20.30479)","(0.8333, 0.16994)","(1, 3.09475)","(0.3333, 1.78879)",NaN,NaN,NaN,NaN,NaN,"(152.4946, 1.4386)","(0.1667, 0.02232)","(0.1667, 0.0515)","(0.5, 0.00372)","(0.1667, 3.85973)",NaN,"(147.4606, 0.77466)",NaN,NaN,NaN,"(Великобритания, 2.60227)","(2, 0.85806)","(Украина, 0.53122)","(Наличные, 7.11875)","(Терминал самообслуживания, 9.7429)","(Наличные, 6.44382)","(Терминал самообслуживания, 14.77418)","(3512164099, 0.0)","(2, 0.23939)","(2, 5.82238)","(2, 0.06465)","(2, 0.08891)","(2, 0.49576)","(2, 0.20628)"
3,"(0.5, 4.6271)","(0.5, 3.90738)","(28.2483, 0.16015)","(4.215, 0.38597)","(1.0, 1.80532

In [8]:
%%time
SCR.describe().toPandas()

CPU times: user 78.9 ms, sys: 19.3 ms, total: 98.2 ms
Wall time: 6min 22s


,summary,cnt_voice_in_fix_avg_6m,cnt_voice_out_fix_avg_6m,arpu_int_roaming_avg_hy,arpu_onnet_roaming_avg_hy,call_charge_avg_6m,call_charge_ind_6m,cnt_voice_out_avg_6m,arpu_voice_avg_hy,arpu_gprs_avg_hy,arpu_vas_avg_hy,ingoing_avg_6m,intercity_in_avg_6m,int_roam_in_avg_6m,sms_out_comp_num_avg_6m,arpu_wo_roam_avg_hy,onn_roam_out_time_6m,cnt_onn_r_in_call_avg_6m,cnt_onn_r_out_call_avg_6m,data_volume_roam_avg_6m,cnt_pp_change_6m,cnt_fin_block_6m,cnt_block_6m,cpa_ind,cnt_package_m,cpa_ind_avg_6m,minus_bal_day_avg_1m,minus_bal_day_avg_hy,conf_call_charge_ind_1m,bl_charge_ind_1m,bal_charge_ind_1m,pay_charge_ind_1m,package_ind_1m,charge_package_1m,conf_call_charge_avg_6m,bl_charge_avg_6m,bal_charge_avg_6m,pay_charge_avg_6m,pay_charge_ind_6m,charge_package_avg_6m,conf_call_charge_ind_6m,bl_charge_ind_6m,bal_charge_ind_6m,int_state_6m,cnt_roam_state_6m,roam_state_6m,payment_method_1m,equipment_type_1m,payment_method_6m,equipment_type_6m,direct_number,cnt_pp_change_1m,cnt_block_1m,cnt_mou_bel_out,cnt_mou_eur_out,cnt_mou_cis_out,cnt_mou_ukr_out
0,count,50582892,50582892,50582892,50582892,50582892,50582892,50582892,50582892,50582892,50582892,50582892,50582892,50582892,50582892,50582892,50582892,50582892,50582892,50582892,50582892,50582892,50582892,50582892,50582892,50582892,50582892,50582892,50582892,50582892,50582892,50582892,50582892,50582892,50582892,50582892,50582892,50582892,50582892,50582892,50582892,50582892,50582892,50582892,50582892,50582892,50582892,50582892,50582892,50582892,50582892,50582892,50582892,50582892,50582892,50582892,50582892
1,mean,2.839635682244162,4.367013416785318,7.547871794422115,11.920980140003062,0.057701963612520474,0.12004989750289485,84.80526871458028,91.57654609789566,80.52409834649164,129.96035649039968,147.41485966673454,21.864382686569247,0.10447228241714614,9.020038058436576,268.4836482189032,72.91586545570965,3.8465851199473,5.037115736101086,0.0,0.3122747311482309,9.620706186589731E-4,0.40120765404637265,0.05275437790310605,2.4638882055221356,0.05173811122740857,5.822954073088585,5.764094042007356,7.43947182774761E-4,0.002357931610553228,6.417189432347996E-5,0.10661430746189839,0.46468964249809996,112.85292348470024,7.823077237260368E-4,0.002292391004452653,8.907336100909394E-5,0.09874221187669475,0.17338988842314512,106.2697008665225,0.0011803793266703692,0.0032798836412912097,2.558967960946163E-4,None,0.08659868637008734,None,None,None,None,None,5.820486538020696E9,0.10007308399843963,0.4569827086992179,0.016112423939698823,0.02075047429079381,1.2708756351851136,0.16993053698867197
2,stddev,12.502121728521443,66.48273519673052,217.9696861422826,102.09726869282355,0.18855902542076514,0.3250198789318146,399.5196737166439,472.8673665206855,542.4227811755355,7283.016304073316,233.80245027839638,85.63421728697082,3.142068035937087,56.45911084033706,4123.127402529566,545.823532122604,21.233114474772908,42.51771345910807,0.0,0.577024005385326,0.01920968107458769,0.680588940359332,0.22354273529478108,1.6844760264952048,0.219704709444235,10.269065712379499,9.129538433827975,0.027265247845206637,0.048501255814447426,0.008010479234104886,0.30862225906327695,0.4987516251310079,207.6118036571304,0.025510353544374906,0.04421211260053266,0.007270263017669836,0.252578165594959,0.37858399048195185,339.1966735288328,0.03433636635731183,0.05717627190732089,0.01599472781831655,None,0.45062482478677895,None,None,None,None,None,2.2897473541741366E9,0.31133131752336785,0.8807722589730066,0.6540530091834503,1.4618008039308987,10.178258542223247,3.3824918989346044
3,min,0.0,0.0,0.0,-0.036,0.0,0,0.0,0.0,-42.387,-252.1582,-136400.3,-10911.3444,0.0,0.0,-141649.0433,-224292.7,0.0,0.0,0.0,0,0.0,0.0,0,0,0.0,0,0.0,0,0,0,0,0,-1115.3595,0.0,0.0,0.0,0.0,0,-174.5787,0,0,0,,0,,,,,,,0,0,0,0,0,0
4,max,21216.25,232108.0,688133.83,333981.75,6.0,1,1391290.8333,1508779.7233,2918064.11,9989207.5533,159572.7639,19112.0111,2558.8194,154986.3333,7872659.5233,1017636.4333,9159.0,104309.3333,0.0,75,2.0,15.0,1,28,6.0,31,31.0,1,1,1,1,1,69038.4843,1

In [16]:
DummyList = [u'NOT_IND', u'NO_VALUE']

## 3. Main cycle

In [17]:
SCR.unpersist();

In [18]:
t0 = time()
writer = pd.ExcelWriter(XlsFileName+'.xlsx')

for mmonth in month_list:
    SCR = sqlContext.sql('select * from %s where month in (%s)' %(ttable, mmonth))\
                    .select(*flist)    
                    #.limit(10000)
    rec_count = SCR.count()
    SCR.persist(storageLevel=StorageLevel.DISK_ONLY) #MEMORY_AND_DISK
    
    print 'Month: ', mmonth, '| rec_count: ', rec_count, '| time: ', np.round((time()-t0)/60.,3), 'mins'
    statDF = [] 
    HTypes = pd.DataFrame(SCR.dtypes, columns=('Column', 'Type'))
    
    for ncol in SCR.columns:
        #print '      ', ncol, np.round((time()-t0)/60.,3)
        
        # 1.NullPers
        NullPers = 100*(SCR.filter(SCR[ncol].isNull()).count() 
                        +  SCR.filter(SCR[ncol]=='')
                        .count()) / rec_count    

        # 2.NullPers with dummy    
        NullPersWithDummy = 100*(SCR.filter(SCR[ncol].isNull()).count() + \
                               SCR.filter(SCR[ncol]=='').count() + \
                               SCR.filter(SCR[ncol].isin(DummyList)).count()\
                              ) / rec_count
            
        # 3. Current type in hive
        HiveType = HTypes[HTypes.Column == ncol].Type.values[0]
        
        # 4. Values
        # for numeric - values range
        # for cat - cats count
        
        if HiveType in ['double','int']:
            ColValues = str(SCR.agg({ncol: "min"}).collect()[0][0]) +' : ' + str(SCR.agg({ncol: "max"}).collect()[0][0])
        else:
            ColValues = SCR.select(ncol).distinct().count()
         
        # 5. Size
        if NullPers > 0:
            ColSize = SCR.where((col(ncol).isNotNull()) & (col(ncol) !='' ))\
                        .select(mean(length(col(ncol)))).collect()[0][0]
        else:
            ColSize = SCR.select(mean(length(col(ncol)))).collect()[0][0]
            
        statDF.append([ncol, 
                       np.round(NullPers,5),
                       np.round(NullPersWithDummy,5),
                       HiveType,
                       ColValues,
                       #np.round(ColSize,5), 
                       np.round(rec_count*(1-NullPers/100)*(ColSize+1)/1024, 2)])    
    
    SCR.unpersist();
    res = pd.DataFrame(statDF,columns=('Column', 'PersOfNulls', 'PersOfNullsWithDummy','TypeInHive','ColValues', 'ColSizeInMBytes'))
    res.to_excel(writer,mmonth)
writer.save()
#Month:  201608 | rec_count:  50582892 | time:  2.022 mins
#Month:  201611 | rec_count:  50060147 | time:  28.881 mins

Month:  201608 | rec_count:  50582892 | time:  3.863 mins
Month:  201611 | rec_count:  50060147 | time:  55.907 mins
Month:  201702 | rec_count:  49171007 | time:  107.894 mins
Month:  201705 | rec_count:  50370283 | time:  181.935 mins
